In [2]:
import numpy as np
from graphviz import Digraph
import itertools as it
from functools import reduce
import sys

#### Перевірка на ациклічність пошуком в глибину

In [22]:
def acyclicity(matrix): 
    size = len(matrix)
    oppos_matrix = opposite(matrix)
    current_matrix = matrix[:]
    if intersection(matrix, oppos_matrix).any():
        return False
    compos_matrix=composition(matrix,matrix)
    while((compos_matrix == current_matrix).all()):
        matrix = compos_matrix[:]
        compos_matrix = composition(compos_matrix,matrix)
        if intersection(matrix,oppos_matrix).any():
            return False
    return True

def intersection(matrix1, matrix2):
    size = len(matrix1)
    intersection_matrix = np.empty((size,size))
    for i in range(size):
        for j in range(size):
            intersection_matrix[i][j] = matrix1[i][j] and matrix2[i][j]
    return intersection_matrix

def composition(matrix1, matrix2): 
    size = len(matrix1)
    compos_matrix = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if matrix1[i][j] == 1:
                for k in range(size):
                    if matrix2[j][k] == 1:
                        compos_matrix[i][k] = 1
    return compos_matrix

def opposite(matrix): 
    return np.transpose(matrix)


no


#### k-оптимізація

In [11]:
def vec_nonzero(a):
    return [set(ai.nonzero()[0]) for i, ai in enumerate(a)]

#асиметрична частина
def P(r):
    return vec_nonzero(r * np.logical_not(r).transpose() * np.logical_not(np.eye(len(r))))
#симетрична частина
def I(r):
    return vec_nonzero(r * r.transpose() + np.eye(len(r)))
#відношення непорівнюваності
def N(r):
    return vec_nonzero(np.logical_not(r) * np.logical_not(r).transpose() * np.logical_not(np.eye(len(r))))

def sk_sets(r):
    p = P(r)
    i = I(r)
    n = N(r)
    #  k=1 : p[j] | i[j] | n[j]
    #  k=2 : p[j] | n[j]
    #  k=3 : p[j] | i[j]
    #  k=4 : p[j]
    return [[p[j] | i[j] | n[j], p[j] | n[j], p[j] | i[j], p[j]] for j in range(0, len(r))]

def get_i_max(Ski):
    a = set()
    for i, i_row in enumerate(Ski):
        for j, j_row in enumerate(Ski):
            if i_row < j_row:
                break
        else:
            a.add(i)
    return a

def k_max(Sk):
    return [get_i_max(Sk[:, i]) for i in range(0, 4)]


#### Оптимізація за Нейманом-Моргенштерном

In [12]:
def nonzero_in_column(r, idx):
    return set(r[:, idx].nonzero()[0])

def s_sets(r):
    S = [set()]
    while S[-1] != set(range(0, len(r))):  
        S.append(set([i for i in range(0, len(r)) if nonzero_in_column(r, i) - set([i]) <= S[-1]]))
    return S

def neuman_morgenstern(r):
    C = s_sets(r)
    elements = np.hstack(map(lambda i: list(C[i] - C[i-1]), range(1, len(C))))
    return reduce(lambda s, i: s | {i} if not nonzero_in_column(r, i) & s else s, elements, set()) 
    

#### Читання та візуалізація графів

In [5]:
with open ("relations.txt", 'r') as data: 
    relations = [[int(x) for x in line.split()] for line in data]
    
make_R = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
Relation = make_R(relations, 15)

In [ ]:
for i, rel in zip(range(len(Relation)),Relation):
    current_relation = np.array(rel)
    print("Відношення R{}:".format(i+1))
    if acyclicity(current_relation):       
        print ("--Ациклічне")
        print ("--Оптимізація за Нейманом-Моргенштерном:{}".format(print_result(neuman_morgenstern(current_relation))))
    else:
        print ("--Містить цикли")
        a = np.array(sk_sets(current_relation))
        print ("--Множини k-оптимальних альтернатив:")
        print ([print_result(x) for x in k_max(a)])
    print ("")

Відношення R1:
--Ациклічне
--Оптимізація за Нейманом-Моргенштерном:{1, 2, 5, 9}

Відношення R2:
--Містить цикли
--Множини k-оптимальних альтернатив:
['{3, 4, 5, 10, 11, 12}', '{3, 4, 5, 10, 11, 12}', '{3, 4, 5, 10, 11, 12}', '{3, 4, 5, 10, 11, 12}']

Відношення R3:
--Ациклічне
--Оптимізація за Нейманом-Моргенштерном:{0, 2, 5}

Відношення R4:
--Ациклічне
--Оптимізація за Нейманом-Моргенштерном:{0, 1, 4}

Відношення R5:
--Ациклічне
--Оптимізація за Нейманом-Моргенштерном:{0, 8, 3}

Відношення R6:
--Містить цикли
--Множини k-оптимальних альтернатив:
['{1, 3, 8, 11, 12, 13}', '{1, 3, 8, 11, 12, 13}', '{1, 3, 8, 11, 12, 13}', '{8, 3, 12}']

Відношення R7:
--Ациклічне


c:\users\teraz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


In [7]:
def print_set(s):
    return "{" + ", ".join(str(v) for v in list(s)) + "}"

def print_result(s):
    return print_set(s)
